# Preface
A lot of this project was inspired by videos from 3Blue1Brown. Along with that, this will come heavily inspired from Neural Networks and Deep Learning, the free online book. It can be reached [neuralnetworksanddeeplearning.com](http://neuralnetworksanddeeplearning.com/) 

In [10]:
# Imports
import random
import numpy as np
from itertools import permutations

In [36]:
class Network(object):
    """
    Creates a Neural Network in specified layers with implemented Neural Network Methods
    """
    def __init__(self, sizes : list[int]):
        """
        Initializes the Network with random, normally distributed, biases and weights.

        Args:
            sizes (list): List of the number of neurons in each layer
        """
        self.sizes = sizes
        self.biases = [np.random.randn(layer, 1) for layer in sizes[1:]]
        self.weights = [np.random.randn(layer, prev) for prev, layer in zip(sizes[:-1], sizes[1:])] 
    
    
    
    def feedForward(self, inputs : list[np.ndarray]):
        """
        Returns the output of the network given the input list

        Args:
            inputs (list): Input to the lyaer
        """
        for bias, weight in zip(self.biases, self.weights):
            inputs = np.tanh(np.dot(weight, inputs) + bias)
        return inputs
    
    
    
    def SGD(self, training_data : list[tuple[np.ndarray, np.ndarray]], epochs : int, mini_batch_size : int, learningRate : float, test_data : list[tuple[np.ndarray, int]] =None) -> None:
        """
        Trains the network using Stoachastic Gradient Descent. 

        Args:
            training_data (list of tuples): Training Data for the network
            epochs (int): number of iterations to adjust network
            mini_batch_size (int): size of each batch to pass through
            learningRate (float): Amount to shift layers in backpropogration
            test_data (list, optional): Passed through test data. Defaults to None.
        """
        
        for i in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[j:j + mini_batch_size] for j in range(0, len(training_data), mini_batch_size)]
            
            for batch in mini_batches:
                self.update_mini_batch(batch, learningRate)
                
            if test_data:
                print(f'Epoch {i}: {self.evaluate(test_data)} / {len(test_data)}') 
            else:
                print(f'Epoch {i} complete!')
        
        
        
    def update_mini_batch(self, mini_batch : list[tuple[np.ndarray, np.ndarray]], learningRate : float):
        """
        Updates the weights and biases using SGD and Back Propogation

        Args:
            mini_batch (list[tuple[np.ndarray, np.ndarray]]): list containing a subset of the training data
            learningRate (float): Amount by which to shift existing layers after backpropogration
        """
        new_biases = [np.zeros(b.shape) for b in self.biases] #array of zeros w/ same dimension [( 1, 2), (3, 4)] -> [(0, 0), (0, 0)] 
        new_weights = [np.zeros(w.shape) for w in self.weights]
        
        for x, y in mini_batch:
            change_new_biases, change_new_weights = self.backprop(x, y)
            new_biases = [cur + change for cur, change in zip(new_biases, change_new_biases)]
            new_weights = [cur + change for cur, change in zip(new_weights, change_new_weights)]
        
        self.weights = [weight - (learningRate / len(mini_batch) * new_weight) for weight, new_weight in zip(self.weights, new_weights)]
        self.biases = [bias - (learningRate / len(mini_batch) * new_biases) for bias, new_biases in zip(self.biases, new_biases)]
        
        
    def backprop(self, x, y) -> tuple:
        """
        Return a tuple representing the gradient for the cost function. 

        Args:
            x (np.ndarray): Input array
            y (np.ndarray): Expected output array
        """
        new_biases = [np.zeros(b.shape) for b in self.biases] #array of zeros w/ same dimension [( 1, 2), (3, 4)] -> [(0, 0), (0, 0)] 
        new_weights = [np.zeros(w.shape) for w in self.weights]
        
        #feed forward
        activation = x
        activations = [x] #To store all activations in each layer
        zs = [] #To store all z vectors
        
        # Iterate across layers
        for bias, weight in zip(self.biases, self.weights):
            z = np.dot(weight, activation) + bias
            zs.append(z)
            activation = np.tanh(z)
            activations.append(activation)
        
        # backwards pass
        delta = self.cost_derivative(activations[-1], y) * (1 / np.cosh(zs[-1])) ** 2 # Find the cost of our result
        new_biases[-1] = delta
        new_weights[-1] = np.dot(delta, activations[-2].transpose())
        
        for layer in range(-2, -len(self.sizes), -1): # Using negative indexing
            z = zs[layer]
            sp = np.tanh(z)
            delta = np.dot(self.weights[layer + 1].transpose(), delta) * sp
            new_biases[layer] = delta
            new_weights[layer] = np.dot(delta, activations[layer - 1].transpose())
            
        return (new_biases, new_weights)
    
    def evaluate(self, test_data : list[tuple[np.ndarray, int]]):
        """
        Return the number of correct outputs

        Args:
            test_data (list[tuple[np.ndarray, np.ndarray]]): test data in the form of ([input], output)
        """
        test_results = [(np.argmax(self.feedForward(x)), y) for (x,y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
        
    
    def cost_derivative(self, output_activations, y):
        """
        Return vector of partial derivatives of cost with respect to layer for the 

        Args:
            output_activations (_type_): what we got
            y (_type_): what the goal was
        """
        return output_activations - y

In [78]:
# Lets test the network!
myNet = Network([4, 10, 10, 4]) # have 4 neurons, 4 neurons, 2 neurons.

# Here is my rule: numbers must be in order. Output should be how many numbers are in order by the end. Simple O(n) 
def numsInPlace(nums : tuple[int]) -> int:
    return sum([1 if i + 1 == nums[i] else 0 for i in range(len(nums))])

assert(numsInPlace(tuple([1, 3, 2, 4])) == 2)

myData = [(np.array(x), np.array([1 if numsInPlace(x) == i else 0 for i in range(4)])) for x in list(permutations([1, 2, 3, 4], 4))]
testData = [(np.array(x), numsInPlace(x) - 1) for x in list(permutations([1, 2, 3, 4], 4)) if random.random() < 0.3]
myData = myData
#self, training_data : list[tuple[np.ndarray, np.ndarray]], epochs : int, mini_batch_size : int, learningRate : float, test_data : list[tuple[np.ndarray, int]] =None
len(myData)
myNet.SGD(myData, 5, 5, 1, test_data=testData)



ValueError: operands could not be broadcast together with shapes (4,10) (4,) 

[array([[ 2.58864509, -5.83889952, -6.74733774, -7.08900456],
        [-0.66149948, -7.95813323, -5.96480889, -5.95328965],
        [ 0.42798111, -6.48215369, -5.00100901, -7.44021706],
        [ 0.56384745, -6.39946471, -6.21058701, -6.63050599]]),
 array([[-2.31627021, -0.83700805, -0.27924348, -0.27206301],
        [-1.65891698, -1.1965343 , -0.50210456, -0.74455794],
        [-0.01145002, -1.60504269,  1.48812526,  0.5795122 ],
        [-1.73423454, -1.54485724, -0.19229157, -0.12170709]])]